In [73]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
from selenium.webdriver.common.by import By

In [2]:
#웹트라이버 자동다운로드
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service

def set_chrome_driver():
    chrome_options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    return driver

In [44]:
browser=set_chrome_driver()



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [C:\Users\duacn\.wdm\drivers\chromedriver\win32\96.0.4664.45\chromedriver.exe] found in cache


In [45]:
browser.get("https://www.youtube.com/watch?v=B4vUIRme_KQ") # 찾을 영상 링크

In [46]:
def infinite_loop():
    #무한 스크롤 내리기
    last_page_height = browser.execute_script("return document.documentElement.scrollHeight")

    while True:
        browser.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(1.0)
        new_page_height = browser.execute_script("return document.documentElement.scrollHeight")
        if new_page_height == last_page_height:
            time.sleep(1.0)
            if new_page_height == browser.execute_script("return document.documentElement.scrollHeight"):
                break
        else:
            last_page_height = new_page_height

In [60]:
infinite_loop()

In [76]:
comments = browser.find_elements(By.CSS_SELECTOR, "#content-text")

In [77]:
len(comments)

2020

In [78]:
text_df=pd.DataFrame(index=range(len(comments)), columns=['text'])
for i in range(len(comments)):
    text_df.loc[i,'text']=comments[i].text

In [79]:
text_df

,text
0,코로나 때문에 다들 고생많다\n댓글에 코로나 끝나면 하고 싶은 것들 달아줘라!\n\...
1,밖에 나가서 마스크 쓰지 않고 편하게 지내는 것만으로도 소원이에요ㅜㅜ
2,우리학교 쌤들은 복도에서 친구들이\n마스크 안쓰고 있으면 화내면서\n쓰라고 하시는데...
3,하 진짜 형은 어떻게 저런 짤들을 찾는거야 맨날 신기하네 ㅎㅎㅋㅋ
4,콘서트랑 코노 진짜 진심 넘 가고싶다\n엄마가 코로나만 끝나면 콘서트 보내준다고 했...
...,...
2015,Top50은 처음아닌가?
2016,처음에 TOP50 이라고해서 헐..... 대박.. 이러면서 들어왔는데.... 아하?
2017,다전부진짜
2018,약 15개


In [93]:
from konlpy.tag import Okt
from kss import split_sentences
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
from pykospacing import Spacing
from hanspell import spell_checker
import re
from textrankr import TextRank
from collections import Counter

In [82]:
# 의미없는 데이터 정제
text_df['text_re']=np.nan
for i in range(len(text_df)):
    try:
        norm = re.compile('[가-힣|0-9|.|?|a-z|A-Z]+').findall(text_df.loc[i,'text'])
        st = ''.join(norm)
        st = spell_checker.check(st)
        space=Spacing(rules=['코노'])
        st2=space(st[2])
        text_df.loc[i,'text_re']=st2
    except:
        print(i)

In [85]:
text_df.tail(20)

,text,text_re
2000,다 공감,다 공감
2001,이미 다들 하시는것 같던데\n마스크도 안쓰시고 말입니다..,이미 다들 하시는 것 같던데 마스크도 안 쓰시고 말입니다..
2002,1:11 카공족 극혐 ㅠㅠㅋㅋㅋㅋ 자리 기본1시간 넘기고 ㅋㅋㅋ,111 카공족극 혐자리 기본 1시간 넘기고
2003,"집콕,p방....\n나는 프로방콕러다!!",집콕p방....나는 프로방콕러다
2004,노오오오오오오오오래ㅐㅐㅐㅐㅐ바ㅏㅏㅏㅏㅏㅏㅏㅇ 가고싶다,노오오오오오오오오래 바가고 싶다
2005,TOP50ㄷㄷㄷ,TOP 50
2006,코로나 죽이고싶긩~~^^,코로나 죽이고 싶긩
2007,입시덕후님 영상보고 잘보고있는데 틱톡광고나옴 극혐;;;,입시 덕후님 영상 보고 잘 보고 있는데 틱톡 광고나 옴극혐
2008,졸업 사진은 오늘 찍고 왔는디..?,졸업사진은 오늘 찍고 왔는데..?
2009,수학여행가기..,수학여행 가기..


In [94]:
# 명사 추출
okt=Okt()
all_text=''
for i in range(len(text_df)):
    all_text+=text_df.loc[i,'text_re']+' '
nouns=okt.nouns(all_text)
conut=Counter(nouns)

In [98]:
noun_list=conut.most_common(100)

In [99]:
noun_list

[('코로나', 475),
 ('고', 366),
 ('마스크', 308),
 ('노래방', 250),
 ('안', 200),
 ('친구', 197),
 ('진짜', 178),
 ('거', 170),
 ('학교', 156),
 ('못', 141),
 ('나', 137),
 ('집', 128),
 ('코노', 127),
 ('개', 124),
 ('때', 108),
 ('것', 107),
 ('이', 97),
 ('기', 94),
 ('수학여행', 91),
 ('난', 90),
 ('내', 86),
 ('전', 81),
 ('저', 81),
 ('애', 71),
 ('사람', 69),
 ('지금', 64),
 ('그냥', 64),
 ('가기', 64),
 ('게', 61),
 ('때문', 59),
 ('추천', 54),
 ('우리', 54),
 ('수', 53),
 ('좀', 53),
 ('번', 53),
 ('중', 53),
 ('시간', 52),
 ('워터파크', 51),
 ('반', 51),
 ('시험', 45),
 ('보고', 45),
 ('데', 45),
 ('콘서트', 44),
 ('덕후', 44),
 ('말', 42),
 ('제', 42),
 ('방', 42),
 ('요', 41),
 ('취소', 40),
 ('뭐', 38),
 ('코로', 38),
 ('노래', 38),
 ('학년', 38),
 ('입시', 37),
 ('해당', 36),
 ('주제', 35),
 ('제발', 35),
 ('생각', 34),
 ('피시방', 34),
 ('영상', 33),
 ('땜', 33),
 ('왜', 32),
 ('더', 32),
 ('듯', 31),
 ('줄', 31),
 ('건', 29),
 ('코', 28),
 ('여행', 28),
 ('후', 28),
 ('걸', 27),
 ('선생님', 26),
 ('연애', 26),
 ('공기', 26),
 ('공원', 26),
 ('중국', 26),
 ('미세먼지', 26),
 ('그', 25),
 ('탑', 25

In [100]:
# 친구 명사 있는 문장만 따오기
friends=[]
okt=Okt()
for i in range(len(text_df)):
    nouns=okt.nouns(text_df.loc[i,'text_re'])
    if '친구' in nouns:
        friends.append(text_df.loc[i,'text'])

In [101]:
friends

['우리학교 쌤들은 복도에서 친구들이\n마스크 안쓰고 있으면 화내면서\n쓰라고 하시는데 정작 교무실 가니까\n나이든 쌤들이 마스크 벗고 있으니까',
 '이기적인 친구들은 뭐 ㅋㅋㅋ 다 지금 하고 있을 듯 ㅋㅋ',
 '내가 시바 노래 쥰나 부르고 싶어서 코노갈까 생각했다가 의료진들 생각해서 집에서 동생한테 욕처먹으면서 노래 불렀는데 \n친구가 노래방 간거 스토리에 올렸을때 진짜 그때 그 허탈감은 이로 말할 수 없다 진짜',
 '코로나 끝나면 노래방..꼭 ..가고싶..다.. 시험 막 끝나고 나서 친구들이랑  이번에 개봉한 공포영화도 보고싶고... 방학때 계곡도 가고.....',
 '입시덕후님....저 정말 미치겠어요.....ㅜㅠㅠ저 광주 사는데 저희 동네 사우나에서 코로나 확진자 많이 나와서 지금 수업도 원격수업으로 좀 하다가 등교수업으로 한다는데 등교수업으로 바뀔때 졸업사진 찍는다는건 좋은거지만 계속 확진자가 늘어나고 있어서 친구들이랑 당분간 못놀고 18일에 친구생일이어서 같이 앵무새 카페 가기로 했는데 코로나 때문에 그냥 친구집에서 한다고 합니다....진짜 코로나가 모든것을 빼앗아갔어요...',
 '1. 피시방에서 롤 \n2. 학교컴실에서 몰래 유튜브보기\n3. 미친듯이 밖에서 "꺄항하" 거리며 뛰어다니기\n4. 친구들이랑 배구&축구 하기',
 '제발 진짜 친구들이랑 모여서 피자나 치킨먹고싶다 코로나야 제발 부탁이다 끝나줘',
 '에버랜드,롯데월드,캐리비안베이,스파밸리, 아무데나 바다 보이는 카페가서 친구랑 셀카 찍기,하동가기,보성가기,마스크 벗고 밖에 돌아다니기,가족 여행 가기 전부 다~ 하고 싶다',
 '저는 친구들과 모여서 옹기 종기 해리포터를 보고싶네요~^^',
 '아진짜ㅠㅠㅠㅠㅠ 중학교 가서 친구랑 교복데이트하기로했는데ㅠㅠㅠㅠㅠ 진짜 코로나 때매ㅠㅠㅠㅠ 교복 덷트 코스 둘다 열심히 짜놨는데ㅠㅠㅠㅠㅠ 진짜 짱친이 멀리 이사가서 못보는데ㅠㅠㅠㅠ 코로나 빨리 끝나라ㅠㅠㅠㅠㅠ',
 '진짜 빨리끝낼려면  나가지말고(최대한노력) 모두를 위해서 노래방이나 클럽은 

In [86]:
# Q_data tf-idf 사용후 keyword 추출
vectorizer = TfidfVectorizer()
sp_matrix = vectorizer.fit_transform(text_df['text_re'])

word2id = defaultdict(lambda : 0)
for idx, feature in enumerate(vectorizer.get_feature_names()):
    word2id[feature] = idx

Q_tfidf = []
for i, sent in enumerate(text_df['text_re']):
    tfidf = sorted([ (token, sp_matrix[i, word2id[token]])for token in sent.split()],
           key = lambda x: x[1] , reverse = True)
    Q_tfidf.append(tfidf)

# Q_tfidf 중복 제거 후 가중치 순으로 정렬
Q_id = [sorted(list(set(lst)), key = lambda x:x[1], reverse =True) for lst in Q_tfidf]

In [89]:
Q_id

[[('잊지', 0.32273037441456287),
  ('추천도', 0.32273037441456287),
  ('달아줘라', 0.32273037441456287),
  ('많다', 0.32273037441456287),
  ('고생', 0.3062044092608752),
  ('것들', 0.2944790503838532),
  ('댓글', 0.2779530852301656),
  ('말고', 0.24970176119945595),
  ('다들', 0.24643937691196063),
  ('코로나', 0.21882356061372324),
  ('주제', 0.21101909316878673),
  ('싶은', 0.18478924350074924),
  ('때문에', 0.18478924350074924),
  ('하고', 0.15386430247351135),
  ('끝나면', 0.13482935794206755),
  ('에', 0.0)],
 [('쓰지', 0.3971177879828391),
  ('소원이에요', 0.3971177879828391),
  ('내는', 0.3767826870862506),
  ('것만으로도', 0.3511634897937599),
  ('않고', 0.3420196024927616),
  ('편하게', 0.3342885559368918),
  ('나가서', 0.2960653043036824),
  ('밖에', 0.2791903704468143),
  ('마스크', 0.158068847660839),
  ('지', 0.0)],
 [('쓰라고', 0.27114342641658906),
  ('화내면서', 0.27114342641658906),
  ('하시는데', 0.27114342641658906),
  ('가니까', 0.27114342641658906),
  ('선생님들은', 0.25725905986218456),
  ('정작', 0.25725905986218456),
  ('있으니까', 0.2572590598621845